<a href="https://colab.research.google.com/github/leon654321/DEEP/blob/main/digit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.datasets import mnist
import numpy as np

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Kaggle/digit-recognizer/train.csv")
test = pd.read_csv("/content/drive/My Drive/Kaggle/digit-recognizer/test.csv")

Y_train = train["label"] #defining labels as Y_train
X_train = train.drop(labels = ["label"],axis = 1) #defining the images as X_train

In [ ]:
(x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()

train1 = np.concatenate([x_train1, x_test1], axis=0)
y_train1 = np.concatenate([y_train1, y_test1], axis=0)

Y_train1 = y_train1
X_train1 = train1.reshape(-1, 28*28)

In [ ]:
import seaborn as sns
g = sns.countplot(Y_train)

In [ ]:
X_train = X_train / 255.0
#X_train = X_train.astype('float32')
X_test = test / 255.0
#X_test = X_test.astype('float32')
X_train1 = X_train1 / 255.0
#X_train1 = X_train1.astype('float32')

# Reshape Picture is 3D array (height = 28px, width = 28px , canal = 1)
X_train = np.concatenate((X_train.values, X_train1))
Y_train = np.concatenate((Y_train, Y_train1))


X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

"""
X_train = X_train.values.reshape(X_train.shape[0],28,28,1)
X_test = X_test.values.reshape(X_test.shape[0],28,28,1)
"""

Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
for i in range(0,10):
    g = plt.imshow(X_train[i][:,:,0]) #displaying random image from the dataset
    plt.show()

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        rotation_range=5,
        zoom_range = 0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,)
        #zca_whitening=True)

datagen.fit(X_train)



In [ ]:
nets = 17
model = [0] *nets
optimizer=Adam(lr=0.001)
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size=3,padding="same",input_shape=(28, 28,1), activation = 'relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size=3, activation = 'relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size=5, strides=2, activation = 'relu'))
    #model[j].add(MaxPool2D(2,2))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size=3,padding="same",activation = 'relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size=3,activation = 'relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size=3,activation = 'relu'))
    #model[j].add(MaxPool2D(2,2))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    """
    model[j].add(Conv2D(128, kernel_size=3,padding="same", activation = 'relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(128, kernel_size=3,activation = 'relu'))
    model[j].add(BatchNormalization())"""
    model[j].add(Conv2D(128, kernel_size=3,activation = 'relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))


    model[j].add(Flatten())
    model[j].add(Dense(256, activation = "relu"))
    model[j].add(BatchNormalization())
    #model[j].add(Dropout(0.4))
    #model[j].add(Dense(128, activation = "relu"))
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation = "softmax"))
    model[j].compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model[0].summary()

In [ ]:
results = np.zeros( (X_test.shape[0],10) ) 
for i in range(nets):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.1)
    #early_stopping =  EarlyStopping(monitor='val_loss',min_delta=0.0,patience=4,)
    model_try = model[i].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=32),
                                  epochs = 8, validation_data = (X_val2,Y_val2),
                                  verbose = 1, steps_per_epoch=300)#,callbacks=[early_stopping])
    results = results + model[i].predict(X_test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submit = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
300/300 [==============================] - 38s 32ms/step - loss: 1.0805 - accuracy: 0.6797 - val_loss: 2.8009 - val_accuracy: 0.3999
Epoch 2/8
300/300 [==============================] - 7s 25ms/step - loss: 0.3375 - accuracy: 0.8938 - val_loss: 0.4692 - val_accuracy: 0.8568
Epoch 3/8
300/300 [==============================] - 7s 24ms/step - loss: 0.2416 - accuracy: 0.9264 - val_loss: 0.0857 - val_accuracy: 0.9739
Epoch 4/8
300/300 [==============================] - 7s 24ms/step - loss: 0.1864 - accuracy: 0.9434 - val_loss: 0.0600 - val_accuracy: 0.9816
Epoch 5/8
300/300 [==============================] - 7s 25ms/step - loss: 0.1651 - accuracy: 0.9505 - val_loss: 0.0628 - val_accuracy: 0.9811
Epoch 6/8
300/300 [==============================] - 7s 24ms/step - loss: 0.1339 - accuracy: 0.9599 - val_loss: 0.0489 - val_accuracy: 0.9861
Epoch 7/8
300/300 [==============================] - 7s 24ms/step - loss: 0.1306 - accuracy: 0.9621 - val_loss: 0.0542 - val_accuracy: 0.9835
Epoch

In [ ]:
submit.to_csv("/content/drive/My Drive/Kaggle/digit-recognizer/result16.csv",index=False)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
!nvidia-smi